SPARKSQL을 사용해서 해보았습니다.

먼저 PySpark과 Py4J를 설치하자

In [6]:
!pip install pyspark==3.0.1 py4j==0.10.9 

#### Spark Session 만들기

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("ExampleUsingSparkSQL") \
    .getOrCreate()

In [19]:
spark

#### 애플 주식 CSV 파일 로딩하기: https://pyspark-test-sj.s3-us-west-2.amazonaws.com/appl_stock.csv
일단 pandas 데이터프레임으로 로딩해서 Spark 데이터프레임으로 변경한다

In [140]:
import pandas as pd

apple_pandas_df = pd.read_csv("https://pyspark-test-sj.s3-us-west-2.amazonaws.com/appl_stock.csv")
apple_spark_df = spark.createDataFrame(apple_pandas_df)
apple_spark_df.createOrReplaceTempView("apple")

#### 1> 스키마를 프린트해보기

In [142]:
spark.sql("""
DESC apple
""")

DataFrame[col_name: string, data_type: string, comment: string]

#### 3> 처음 5개의 레코드를 출력해보기

In [136]:
spark.sql("""
SELECT * FROM apple LIMIT 5
""").show()

+----------+----------+----------+----------+----------+---------+------------------+
|      Date|      Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+----------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.380001|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|213.249994|214.379993|150476200|         27.774976|
|2010-01-06|214.379993|    215.23|210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|209.050005|    210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.060005|211.980005|111902700|         27.464034|
+----------+----------+----------+----------+----------+---------+------------------+



#### 5> Close 컬럼의 평균값은 얼마인가?

In [58]:
mean_close_df = spark.sql("""
SELECT MEAN(Close) meanClose
FROM apple
""")

mean_close_df.show()

+-----------------+
|        meanClose|
+-----------------+
|312.9270656379114|
+-----------------+



#### 6> Volume 컬럼의 최대값과 최소값은?

In [134]:
min_max_volume_df = spark.sql("""
SELECT MAX(Volume) maxVolume, MIN(Volume) minVolume
from apple
""")

min_max_volume_df.show()

+---------+---------+
|maxVolume|minVolume|
+---------+---------+
|470249500| 11475900|
+---------+---------+



#### 보너스 질문: HV ratio라는 이름의 새로운 컬럼을 추가한 데이터프레임을 만들기. 이 컬럼의 값은 High/Volume으로 계산된다

In [129]:
apple_spark_df_with_hv = spark.sql("""
SELECT *, High/Volume as hv_ratio
FROM APPLE
""")

apple_spark_df_with_hv.show(5)

+----------+----------+----------+----------+----------+---------+------------------+--------------------+
|      Date|      Open|      High|       Low|     Close|   Volume|         Adj Close|            hv_ratio|
+----------+----------+----------+----------+----------+---------+------------------+--------------------+
|2010-01-04|213.429998|214.499996|212.380001|214.009998|123432400|         27.727039|1.737793286041590...|
|2010-01-05|214.599998|215.589994|213.249994|214.379993|150476200|         27.774976|1.432718223878593...|
|2010-01-06|214.379993|    215.23|210.750004|210.969995|138040000|27.333178000000004|1.559185743262822...|
|2010-01-07|    211.75|212.000006|209.050005|    210.58|119282800|          27.28265|1.777288980473295...|
|2010-01-08|210.299994|212.000006|209.060005|211.980005|111902700|         27.464034|1.894503045949740...|
+----------+----------+----------+----------+----------+---------+------------------+--------------------+
only showing top 5 rows



#### 보너스 질문: 월별 Close 컬럼의 평균값은?

In [128]:
spark.sql("""
    SELECT MONTH(Date) as MONTH, MEAN(Close) meanClose
    FROM apple
    GROUP BY 1
    ORDER BY 1
""").show()

+-----+------------------+
|MONTH|         meanClose|
+-----+------------------+
|    1| 322.2097142571429|
|    2| 321.3595563037037|
|    3| 332.9115673137254|
|    4| 340.5104108150685|
|    5|  351.621020857143|
|    6|288.12546566000003|
|    7| 281.7221621148649|
|    8|300.43858096129026|
|    9| 301.0763195902777|
|   10|308.30552563157903|
|   11| 306.2725174895105|
|   12|302.35053626845644|
+-----+------------------+

